In [39]:
import pandas as pd
import numpy as np
import sys
#Add the ptdraft folder path to the sys.path list
sys.path.append('~⁩/⁨PycharmProjects⁩/Notebooks⁩')
from authentication.CYcreds import google, slack, jira

In [40]:
ads_invoice_by_month_fp = r'Data/PathToPurchase/DataFrameFiles/MergedAdsInvoiceByMonth.csv'
ads_invoice_by_month_df = pd.read_csv(ads_invoice_by_month_fp,sep=',', header=0,parse_dates=[0])
ads_invoice_by_month_df.head()
ads_invoice_by_month_df.dtypes
ads_invoice_by_month_df.shape

ads_invoice_by_month_df.dtypes


Month                       datetime64[ns]
Vendor Medium                       object
LC Vendor Medium                    object
Ads Month                           object
Ads Vendor Medium                   object
LC Ads Vendor Medium                object
Invoice Month                       object
Invoice Vendor Medium               object
LC Invoice Vendor Medium            object
Marketing Invoice                  float64
Ads Spend                          float64
Add To Cart                          int64
Lead Submission                      int64
Whitepaper Download                  int64
Conversions Adjusted                 int64
Total Conversions                    int64
Impressions                          int64
Clicks                               int64
dtype: object

In [82]:
orders_by_month_fp = r'Data/PathToPurchase/DataFrameFiles/OrdersByMonth.csv'
orders_by_month_df = pd.read_csv(orders_by_month_fp,sep=',', header=0,parse_dates=[0])
orders_by_month_df['Order Month'] = pd.to_datetime(orders_by_month_df['Order Month'],format='%Y-%m-%d')
orders_by_month_df['Order Number']=orders_by_month_df['Netsuite Order Number'].astype(int)
orders_by_month_df.shape
orders_by_month_df.head()

orders_by_month_df.dtypes




Netsuite Order Number             object
Order Date                        object
Order Created Date                object
Net Amount                       float64
Amount                           float64
validated Status                  object
Workdays To Fulfill              float64
Medium                            object
Vendor                            object
Marketing Tracking URL            object
Discovery Source                  object
Content                           object
Lead Entry Source                 object
Order Source                      object
Term                              object
OrdersVendorMedium                object
Mapped Vendor                     object
Mapped Medium                     object
Order Month               datetime64[ns]
Order Number                       int64
dtype: object

In [137]:
group_orders_by_month_df = orders_by_month_df.groupby([
                                                'Order Month'
                                                ,'OrdersVendorMedium'
                                                ,'Vendor'
                                                ,'Medium'], as_index=False).agg({
                                                         'Amount': 'sum'
                                                          , 'Net Amount': 'sum'
                                                          , 'Workdays To Fulfill': 'sum'
                                                          , 'Order Number': 'count'
                                                          , 'Netsuite Order Number': (lambda x: ','.join(x))
                                                          
                                                })

group_orders_by_month_df.rename(columns={"OrdersVendorMedium":"Orders Vendor Medium"
                                         ,"Amount": "Order Amount"
                                         , "Net Amount": "Order Net Amount"
                                         , "Workdays To Fulfill": "Workdays To Fulfill"
                                         , "Order Number": "Total Orders"
                                         , "Netsuite Order Number":"Order Numbers CSV"
                                        },inplace=True)
group_orders_by_month_df.head()

,Order Month,Orders Vendor Medium,Vendor,Medium,Order Amount,Order Net Amount,Workdays To Fulfill,Total Orders,Order Numbers CSV
0,2018-01-01,Contact_Us_website,website,Contact Us,22995.0,19997.0,26.0,11,"15862,15867,15888,15897,15905,15910,15913,1591..."
1,2018-01-01,Duro_Lead_Form_website,website,Duro Lead Form,2855.0,2590.0,4.0,2,"15886,15889"
2,2018-01-01,Google_SEM,google,SEM,2710.0,2690.0,5.0,2,"15908,15926"
3,2018-01-01,SEO_Search_Engine_sales@,sales@,SEO Search Engine,2101.0,1995.0,4.0,1,15713
4,2018-01-01,Whitepaper_Downloads_auto,auto,Whitepaper Downloads,2177.0,1995.0,4.0,2,"15869,15878"


In [138]:

merge_ads_invoice_orders_by_month_df= pd.merge(ads_invoice_by_month_df
                                        ,group_orders_by_month_df
                                        ,right_on=['Order Month'
                                                   ,'Orders Vendor Medium'
                                                  ]
                                        ,left_on=['Month'
                                                ,'Vendor Medium'
                                                 ]
                                        ,how='outer'
                                       )
merge_ads_invoice_orders_by_month_df.head()

,Month,Vendor Medium,LC Vendor Medium,Ads Month,Ads Vendor Medium,LC Ads Vendor Medium,Invoice Month,Invoice Vendor Medium,LC Invoice Vendor Medium,Marketing Invoice,...,Clicks,Order Month,Orders Vendor Medium,Vendor,Medium,Order Amount,Order Net Amount,Workdays To Fulfill,Total Orders,Order Numbers CSV
0,2018-01-01,Farm_Equipment_Showcase_Print_Ad,farmequipmentshowcase_printad,2018-01-01 00:00:00,Farm_Equipment_Showcase_Print_Ad,farmequipmentshowcase_printad,2018-01-01 00:00:00,Farm_Equipment_Showcase_Print_Ad,farmequipmentshowcase_printad,1997.01,...,7291.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,GPS_World_Print_Ad,gpsworld_printad,0,0,0,2018-01-01 00:00:00,GPS_World_Print_Ad,gpsworld_printad,3138.22,...,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,Google_GDN_Ad,google_gdnresponsiveads,2018-01-01 00:00:00,Google_GDN_Ad,google_gdnresponsiveads,2018-01-01 00:00:00,Google_GDN_Ad,google_gdnresponsiveads,986.62,...,1004.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,Google_SEM,google_sem,2018-01-01 00:00:00,Google_SEM,google_sem,2018-01-01 00:00:00,Google_SEM,google_sem,5442.80,...,3154.0,2018-01-01,Google_SEM,google,SEM,2710.0,2690.0,5.0,2.0,"15908,15926"
4,2018-01-01,Netline_Lead_Generation,netline_leadgeneration,2018-01-01 00:00:00,Netline_Lead_Generation,netline_leadgeneration,2018-01-01 00:00:00,Netline_Lead_Generation,netline_leadgeneration,3024.10,...,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
merge_ads_invoice_orders_by_month_df.dtypes

Month                       datetime64[ns]
Vendor Medium                       object
LC Vendor Medium                    object
Ads Month                           object
Ads Vendor Medium                   object
LC Ads Vendor Medium                object
Invoice Month                       object
Invoice Vendor Medium               object
LC Invoice Vendor Medium            object
Marketing Invoice                  float64
Ads Spend                          float64
Add To Cart                        float64
Lead Submission                    float64
Whitepaper Download                float64
Conversions Adjusted               float64
Total Conversions                  float64
Impressions                        float64
Clicks                             float64
Order Month                 datetime64[ns]
Orders Vendor Medium                object
Vendor                              object
Medium                              object
Order Amount                       float64
Order Net A

In [140]:
column_drop_list =  ['LC Vendor Medium'
                     ,'LC Ads Vendor Medium'
                     ,'LC Invoice Vendor Medium'
                     ,'Vendor'
                     ,'Medium'
                    ]

merge_ads_invoice_orders_by_month_df.drop(column_drop_list,  axis=1)

,Month,Vendor Medium,Ads Month,Ads Vendor Medium,Invoice Month,Invoice Vendor Medium,Marketing Invoice,Ads Spend,Add To Cart,Lead Submission,...,Total Conversions,Impressions,Clicks,Order Month,Orders Vendor Medium,Order Amount,Order Net Amount,Workdays To Fulfill,Total Orders,Order Numbers CSV
0,2018-01-01,Farm_Equipment_Showcase_Print_Ad,2018-01-01 00:00:00,Farm_Equipment_Showcase_Print_Ad,2018-01-01 00:00:00,Farm_Equipment_Showcase_Print_Ad,1997.01,1997.0,0.0,0.0,...,0.0,0.0,7291.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,GPS_World_Print_Ad,0,0,2018-01-01 00:00:00,GPS_World_Print_Ad,3138.22,0.0,0.0,0.0,...,0.0,0.0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,Google_GDN_Ad,2018-01-01 00:00:00,Google_GDN_Ad,2018-01-01 00:00:00,Google_GDN_Ad,986.62,1000.0,0.0,3.0,...,3.0,311582.0,1004.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,Google_SEM,2018-01-01 00:00:00,Google_SEM,2018-01-01 00:00:00,Google_SEM,5442.80,9728.0,0.0,41.0,...,72.0,222372.0,3154.0,2018-01-01,Google_SEM,2710.0,2690.00,5.0,2.0,"15908,15926"
4,2018-01-01,Netline_Lead_Generation,2018-01-01 00:00:00,Netline_Lead_Generation,2018-01-01 00:00:00,Netline_Lead_Generation,3024.10,3000.0,0.0,0.0,...,100.0,0.0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
5,2018-01-01,PR,0,0,2018-01-01 00:00:00,PR,5650.00,0.0,0.0,0.0,...,0.0,0.0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
6,2018-01-01,Twitter_Social,2018-01-01 00:00:00,Twitter_Social,2018-01-01 00:00:00,Twitter_Social,1499.60,1519.0,0.0,5.0,...,5.0,151676.0,529.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
7,2018-01-01,YahooBing_SEM,2018-01-01 00:00:00,YahooBing_SEM,2018-01-01 00:00:00,YahooBing_SEM,1308.07,992.0,0.0,11.0,...,11.0,6147.0,524.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
8,2018-01-01,eEffective_Retargeting,2018-01-01 00:00:00,eEffective_Retargeting,2018-01-01 00:00:00,eEffective_Retargeting,5100.00,4889.0,0.0,11.0,...,29.0,612665.0,526.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
9,2018-01-01,xyHt_Email,2018-01-01 00:00:00,xyHt_Email,2018-01-01 00:00:00,xyHt_Email,2226.01,2763.0,0.0,0.0,...,0.0,2020.0,109.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
#Change the order of the columns
merge_ads_invoice_orders_by_month_df = merge_ads_invoice_orders_by_month_df[[
                                       'Month'
                                      ,'Order Month'
                                      ,'Ads Month'
                                      ,'Invoice Month'
                                      ,'Vendor Medium'
                                      ,'Ads Vendor Medium'
                                      ,'Invoice Vendor Medium'
                                      ,'Orders Vendor Medium'
                                      ,'Marketing Invoice'
                                      ,'Ads Spend'
                                      ,'Order Amount'
                                      ,'Order Net Amount'
                                      ,'Total Orders'
                                      ,'Add To Cart'
                                      ,'Lead Submission'
                                      ,'Whitepaper Download'
                                      ,'Conversions Adjusted'
                                      ,'Total Conversions'
                                      ,'Impressions'
                                      ,'Clicks'
                                      ,'Workdays To Fulfill'
                                      ,'Order Numbers CSV'
                ]]
merge_ads_invoice_orders_by_month_df

,Month,Order Month,Ads Month,Invoice Month,Vendor Medium,Ads Vendor Medium,Invoice Vendor Medium,Orders Vendor Medium,Marketing Invoice,Ads Spend,...,Total Orders,Add To Cart,Lead Submission,Whitepaper Download,Conversions Adjusted,Total Conversions,Impressions,Clicks,Workdays To Fulfill,Order Numbers CSV
0,2018-01-01,NaT,2018-01-01 00:00:00,2018-01-01 00:00:00,Farm_Equipment_Showcase_Print_Ad,Farm_Equipment_Showcase_Print_Ad,Farm_Equipment_Showcase_Print_Ad,NaN,1997.01,1997.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,7291.0,NaN,NaN
1,2018-01-01,NaT,0,2018-01-01 00:00:00,GPS_World_Print_Ad,0,GPS_World_Print_Ad,NaN,3138.22,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,2018-01-01,NaT,2018-01-01 00:00:00,2018-01-01 00:00:00,Google_GDN_Ad,Google_GDN_Ad,Google_GDN_Ad,NaN,986.62,1000.0,...,NaN,0.0,3.0,0.0,3.0,3.0,311582.0,1004.0,NaN,NaN
3,2018-01-01,2018-01-01,2018-01-01 00:00:00,2018-01-01 00:00:00,Google_SEM,Google_SEM,Google_SEM,Google_SEM,5442.80,9728.0,...,2.0,0.0,41.0,14.0,52.0,72.0,222372.0,3154.0,5.0,"15908,15926"
4,2018-01-01,NaT,2018-01-01 00:00:00,2018-01-01 00:00:00,Netline_Lead_Generation,Netline_Lead_Generation,Netline_Lead_Generation,NaN,3024.10,3000.0,...,NaN,0.0,0.0,0.0,100.0,100.0,0.0,0.0,NaN,NaN
5,2018-01-01,NaT,0,2018-01-01 00:00:00,PR,0,PR,NaN,5650.00,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
6,2018-01-01,NaT,2018-01-01 00:00:00,2018-01-01 00:00:00,Twitter_Social,Twitter_Social,Twitter_Social,NaN,1499.60,1519.0,...,NaN,0.0,5.0,0.0,5.0,5.0,151676.0,529.0,NaN,NaN
7,2018-01-01,NaT,2018-01-01 00:00:00,2018-01-01 00:00:00,YahooBing_SEM,YahooBing_SEM,YahooBing_SEM,NaN,1308.07,992.0,...,NaN,0.0,11.0,0.0,11.0,11.0,6147.0,524.0,NaN,NaN
8,2018-01-01,NaT,2018-01-01 00:00:00,2018-01-01 00:00:00,eEffective_Retargeting,eEffective_Retargeting,eEffective_Retargeting,NaN,5100.00,4889.0,...,NaN,0.0,11.0,0.0,29.0,29.0,612665.0,526.0,NaN,NaN
9,2018-01-01,NaT,2018-01-01 00:00:00,2018-01-01 00:00:00,xyHt_Email,xyHt_Email,xyHt_Email,NaN,2226.01,2763.0,...,NaN,0.0,0.0,0.0,0.0,0.0,2020.0,109.0,NaN,NaN


In [143]:
#TypeError: cannot compare a dtyped [float64] array with a scalar of type [bool]
#& has higher precedence than ==, so use Parantheses for each condition

#ads_invoice_spend_by_month_df = merge_ads_invoice_by_month_df.drop(merge_ads_invoice_by_month_df
#                                [(merge_ads_invoice_by_month_df['Ads Spend'] == 0) & (merge_ads_invoice_by_month_df['Marketing Invoice'] == 0)].index)




In [146]:
merge_ads_invoice_orders_by_month_df.to_csv('Data/PathToPurchase/DataFrameFiles/MergedAdsInvoiceOrdersByMonth.csv', sep=',', encoding='utf-8',index=False )


In [10]:
del merge_ads_invoice_orders_by_month_df